In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Загрузка данных для Московской области
cian1 = pd.read_excel("циан 1 комната.xlsx")
cian2 = pd.read_excel("циан 2 комнаты.xlsx")
cian3 = pd.read_excel("циан 3 комнаты.xlsx")
cian4 = pd.read_excel("циан 4 комнаты.xlsx")
cian5 = pd.read_excel("циан 5 комнат.xlsx")
cian6 = pd.read_excel("циан 6 комнат.xlsx")
cian_new_1 = pd.read_excel("циан новостройка 1.xlsx")
cian_new_2 = pd.read_excel("циан новостройка 2.xlsx")
cian_new_3 = pd.read_excel("циан новостройка 3+.xlsx")
cian = pd.read_excel("циан свободная планировка.xlsx")
cian_st = pd.read_excel("циан студия.xlsx")

# Загрузка данных для Москвы
mos1 = pd.read_excel("Москва 1.xlsx")
mos2 = pd.read_excel("Москва 2.xlsx")
mos3 = pd.read_excel("Москва 3.xlsx")
mos4 = pd.read_excel("Москва 4.xlsx")
mos5 = pd.read_excel("Москва 5.xlsx")
mos6 = pd.read_excel("Москва 6+.xlsx")
mos_st = pd.read_excel("Москва свободная планировка.xlsx")
mos = pd.read_excel("Москва студия.xlsx")

# Загрузка данных для Москвы новостройки
mos_new1 = pd.read_excel('Москва новостройка 1.xlsx')
mos_new2 = pd.read_excel('Москва новостройка 2.xlsx')
mos_new3 = pd.read_excel('Москва новостройка 3.xlsx')
mos_new4 = pd.read_excel('Москва новостройка 4.xlsx')
mos_new5 = pd.read_excel('Москва новостройка 5.xlsx')
mos_new6 = pd.read_excel('Москва новостройка 6+.xlsx')

In [3]:
cian1['Количество комнат'] = 1
cian2['Количество комнат'] = 2
cian3['Количество комнат'] = 3
cian4['Количество комнат'] = 4
cian5['Количество комнат'] = 5
cian6['Количество комнат'] = 6
cian_new_1['Количество комнат'] = 1
cian_new_2['Количество комнат'] = 2
cian_new_3['Количество комнат'] = 3 
cian['Количество комнат'] = 0
cian_st['Количество комнат'] = 0
mos1['Количество комнат'] = 1
mos2['Количество комнат'] = 2
mos3['Количество комнат'] = 3
mos4['Количество комнат'] = 4
mos5['Количество комнат'] = 5
mos6['Количество комнат'] = 6
mos_st['Количество комнат'] = 0
mos['Количество комнат'] = 0
mos_new1['Количество комнат'] = 1
mos_new2['Количество комнат'] = 2
mos_new3['Количество комнат'] = 3
mos_new4['Количество комнат'] = 4
mos_new5['Количество комнат'] = 5
mos_new6['Количество комнат'] = 6

In [4]:
df = pd.concat([cian1,cian2,cian3,cian4,cian5,cian6, 
                cian_new_1,cian_new_2,cian_new_3,cian,cian_st,
                mos1,mos2,mos3,mos4,mos5,mos6,mos_st,mos,
                mos_new1,mos_new2,mos_new3,mos_new4,mos_new5,mos_new6], ignore_index=True)
df.shape

(4741, 23)

# Функция для обработки набора данных Cian

In [5]:
def cian_processing(df):
    """
    Обработка данных, полученных с сайта Cian.
    
    Функция удаляет ненужные признаки ('ID', 'Тип', 'Телефоны', 'Описание') и отбирает признаки, в которых 
    пропущенных значений < 41%.
    
    Затем происходит следующее преобразование:
    1. Удаление ненужных символов, таких как скобки или запятые.
    2. Разделение признаков на несколько, увеличивая размерность набора данных.
    3. Изменение типа данных на float для числовых столбцов ('Площадь', 'Жилая площадь', 'Кухня площадь', 
       'Этаж', 'Всего этажей', 'Цена').
    4. Перестановка столбцов, чтобы 'Цена' оказалась на первом месте.
   
   """
    
    # Удаление ненужных столбцов
    df.drop(['ID', 'Тип', 'Телефоны', 'Описание'], axis = 1, inplace = True)
    
    # Создание таблицы для определения процента пропущенных значений для каждого признака
    data = pd.DataFrame()
    data.index = df.columns
    # Тип данных 
    data['dtypes'] = df.dtypes
    # Количество пропущенных значений
    data['NaN'] = df.isna().sum()
    # Количество непропущенных значений
    data['count'] = df.count()
    # Процент пропущенных значений
    data['NaN%'] = round((data['NaN'] / df.shape[0]) * 100, 2)
    
    # Выборка столбцов у которых < 41% пропусков
    columns = data[data['NaN%'] < 41].index
    # Обновление набора данных
    df = df[columns]
    
    # Удаление ненужных символов
    
    # Метро
    df['Метро'] = df['Метро'].apply(lambda x: str(x).replace(')', ''))
    df['Метро'] = df['Метро'].apply(lambda x: str(x).split(' ('))
    
    # Площадь
    df['Площадь, м2'] = df['Площадь, м2'].apply(lambda x: x.split('/'))
    
    # Дом
    df['Дом'] = df['Дом'].apply(lambda x: x.split(','))
    
    # Цена
    df['Цена'] = df['Цена'].apply(lambda x: x.replace('.',''))
    df['Цена'] = df['Цена'].apply(lambda x: x.split(' руб'))
    
    # Санузел
    df['Санузел'] = df['Санузел'].apply(lambda x: str(x).replace('(', '').replace(')', ''))
    df['Санузел'] = df['Санузел'].apply(lambda x: str(x).split(' '))
    
    # Лифт
    df['Лифт'] = df['Лифт'].apply(lambda x: str(x).replace('(', '').replace(')', ''))
    df['Лифт'] = df['Лифт'].apply(lambda x: str(x).split(','))
    
    # Разделение и преобразование столбца
    
    # Станция метро
    metro_station = list(df['Метро'].apply(lambda x: x[0] if x != ['nan'] else np.nan).values)
    # Вставка нового столбца
    df.insert(1, 'Станция метро', metro_station)
    
    # Сколько минут до метро
    min_metro = list(df['Метро'].apply(lambda x: x[1] if x != ['nan'] else np.nan).values)
    df.insert(2, 'Минут до метро', min_metro)
    
    # Удаление столбца метро
    df.drop('Метро', axis = 1, inplace = True)
    
    # Площадь
    square = list(df['Площадь, м2'].apply(lambda x: x[0] if len(x) >= 1 else np.nan).values)
    df.insert(4, 'Площадь', square)
    
    # Жилая площадь
    living_area = list(df['Площадь, м2'].apply(lambda x: x[1] if len(x) >= 2 else np.nan).values)
    df.insert(5, 'Жилая площадь', living_area)
    
    # Площадь кухни
    kitchen_area = list(df['Площадь, м2'].apply(lambda x: x[2] if len(x) == 3 else np.nan).values)
    df.insert(6, 'Кухня площадь', kitchen_area)
    
    # Удаление столбца Площадь, м2
    df.drop('Площадь, м2', axis = 1, inplace = True)
    
    # Этаж
    floor = list(df['Дом'].apply(lambda x: x[0] if len(x) >= 1 else np.nan).values)
    df.insert(7, 'Этаж', floor)
    
    # Дом
    df['Дом'] = list(df['Дом'].apply(lambda x: x[1] if len(x) >= 2 else np.nan).values)
    
    # Тип продажи
    tupe_of_sale = list(df['Цена'].apply(lambda x: x[1].split(', ')[1] if len(x) > 1 else np.nan).values)
    df.insert(10, 'Тип продажи', tupe_of_sale)
    
    # Цена
    df['Цена'] = df['Цена'].apply(lambda x: x[0])
    
    # Удаление столбца Нащвание ЖК
    df.drop('Название ЖК', axis = 1, inplace = True)
    
    # Санузел
    df['Санузел'] = list(df['Санузел'].apply(lambda x: x[0] if x != ['nan'] else np.nan).values)
    
    # Лифт
    df['Лифт'] = list(df['Лифт'].apply(lambda x: 'есть' if x != ['nan'] else np.nan).values)
    
    # Добавление столбца Всего этажей
    count_floot = list(df['Этаж'].apply(lambda x: x.split('/')[1]).values)
    df.insert(8, 'Всего этажей', count_floot)
    
    # Этаж квартиры
    df['Этаж'] = list(df['Этаж'].apply(lambda x: x.split('/')[0]).values)
    
    # Изменение типа данных
    
    # Оставляет только длину цены меньше 20 (так как есть строки с $ или евро)
    df = df[df['Цена'].apply(len) <= 20]
    df.index = range(0, df.shape[0])
    
    # Замена типа данных
    numeric = ['Площадь', 'Жилая площадь', 'Кухня площадь', 'Этаж', 'Всего этажей', 'Цена']
    for n in numeric:
        df[n] = df[n].astype('float')
        
    # Перестановка столбцов в другом порядке
    columns = ['Цена', 'Количество комнат', 'Станция метро', 'Минут до метро', 'Адрес',
           'Площадь', 'Жилая площадь', 'Кухня площадь', 'Этаж', 'Всего этажей',
           'Дом', 'Парковка', 'Тип продажи', 'Ремонт', 'Окна', 'Санузел',
           'Высота потолков, м', 'Лифт', 'Ссылка на объявление']

    df = df[columns]

    # Преобразование столбца Минут до метро
    
    # Разделяем по пробелу
    df['Минут до метро'] = df[df['Минут до метро'].notna()]['Минут до метро'].apply(lambda x: str(x).split(' '))

    # Создаем Series и записываем число если есть, если None то NaN
    min_metro = df[df['Минут до метро'].notna()]['Минут до метро'].apply(lambda x: x[0] if x[0] != 'None' else np.nan)
    df['Минут до метро'] = min_metro
    
    # Делаем тип данных - float
    df['Минут до метро'] = df['Минут до метро'].astype('float')
    
    # Возвращем обработанный набор данных
    return df

In [6]:
df = cian_processing(df)

C:\Users\Егор\AppData\Local\Temp\ipykernel_4792\2446977427.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Метро'] = df['Метро'].apply(lambda x: str(x).replace(')', ''))
C:\Users\Егор\AppData\Local\Temp\ipykernel_4792\2446977427.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Метро'] = df['Метро'].apply(lambda x: str(x).split(' ('))
C:\Users\Егор\AppData\Local\Temp\ipykernel_4792\2446977427.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [7]:
df.head(3)

,Цена,Количество комнат,Станция метро,Минут до метро,Адрес,Площадь,Жилая площадь,Кухня площадь,Этаж,Всего этажей,Дом,Парковка,Тип продажи,Ремонт,Окна,Санузел,"Высота потолков, м",Лифт,Ссылка на объявление
0,6300000.0,1,м. Опалиха,6.0,"Московская область, Красногорский район, Красн...",30.6,11.1,8.5,25.0,25.0,Монолитный,наземная,Свободная продажа,Косметический,Во двор,Раздельный,NaN,есть,https://www.cian.ru/sale/flat/289238903
1,6850000.0,1,NaN,NaN,"Московская область, Красногорский район, Красн...",35.6,18.0,7.0,16.0,26.0,Монолитный,NaN,Свободная продажа,Без ремонта,NaN,NaN,NaN,есть,https://www.cian.ru/sale/flat/274781081
2,9000000.0,1,м. Павшино,2.0,"Московская область, Красногорский район, Красн...",49.2,20.0,10.0,6.0,15.0,Кирпичный,NaN,Свободная продажа,Евроремонт,NaN,Совмещенный,2.9,есть,https://www.cian.ru/sale/flat/292849723


### Типы данных

In [8]:
df.dtypes

Цена                    float64
Количество комнат         int64
Станция метро            object
Минут до метро          float64
Адрес                    object
Площадь                 float64
Жилая площадь           float64
Кухня площадь           float64
Этаж                    float64
Всего этажей            float64
Дом                      object
Парковка                 object
Тип продажи              object
Ремонт                   object
Окна                     object
Санузел                  object
Высота потолков, м      float64
Лифт                     object
Ссылка на объявление     object
dtype: object

### Процент проаущенных значений

In [9]:
round((df.isna().sum() / df.shape[0]) * 100,2)

Цена                     0.00
Количество комнат        0.00
Станция метро            4.06
Минут до метро           5.84
Адрес                    0.00
Площадь                  0.00
Жилая площадь           29.68
Кухня площадь           45.11
Этаж                     0.00
Всего этажей             0.00
Дом                     14.57
Парковка                40.47
Тип продажи              0.00
Ремонт                  36.46
Окна                    38.35
Санузел                 22.63
Высота потолков, м      29.98
Лифт                    32.09
Ссылка на объявление     0.00
dtype: float64

# Сохранение набора данных

In [10]:
df.to_excel('Cian Москва и Область.xlsx', index = False)